In [ ]:
# Финальный ноутбук: Распознавание эмоций по лицу для мобильного приложения

Этот ноутбук тренирует компактную и точную модель распознавания эмоций, предназначенную для работы на устройстве (TensorFlow Lite / ONNX) и интеграции в Kotlin Multiplatform.

- Датасеты: `train` (папки классов) и `test` (jpg-файлы), ссылки для автозагрузки ниже
- Модель: EfficientNet-Lite0 (или MobileNetV3Small) с дообучением на ваших данных
- Детекция лица: MTCNN или OpenCV DNN (выбор в настройках), кроп лица → классификация эмоций
- Метрики: accuracy, F1-macro, confusion matrix, ROC-AUC per class
- Визуализации: красивые лоссы/метрики, Grad-CAM, t-SNE feature embeddings
- Экспорт: TFLite (float16 или int8) и ONNX (опционально) + примеры вызова из Kotlin

В ноутбуке предусмотрены прогресс-бары и информативные логи этапов, чтобы было понятно, что происходит на каждом шаге.


In [ ]:
# --- Установка и импорт зависимостей (Colab) ---
!pip -q install -U 'tensorflow==2.19.0' onnx onnxruntime-gpu onnxconverter-common tf2onnx mtcnn 'opencv-python-headless==4.10.0.84' 'albumentations==1.4.7' seaborn 'rich<14' kaggle gdown --no-warn-script-location

import os, sys, time, json, random, shutil, zipfile, glob, math, pathlib
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from mtcnn import MTCNN
import cv2

import albumentations as A

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.manifold import TSNE

from rich.console import Console
from rich.progress import track

console = Console()

In [6]:
# --- Конфигурация проекта ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# пути к датасетам в Colab (будут скачаны)
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data'
TRAIN_DIR = DATA_DIR / 'train'
TEST_DIR  = DATA_DIR / 'test'
DATA_DIR.mkdir(exist_ok=True, parents=True)

# классы эмоций (по структуре вашего train)
CLASS_NAMES = ['anger','contempt','disgust','fear','happy','neutral','sad','surprise','uncertain']
N_CLASSES = len(CLASS_NAMES)

# изображение
IMG_SIZE = 224
BATCH_SIZE = 32

# геометрия детектора и выбор
FACE_DETECTOR = 'mtcnn'  # 'mtcnn' | 'cv-dnn'

# обучение
EPOCHS = 20
BASE_LR = 3e-4
WEIGHT_DECAY = 1e-5

# экспорт
DO_TFLITE_FP16 = True
DO_TFLITE_INT8 = False
DO_ONNX = True


In [7]:
# --- Загрузка датасетов из ссылок ---

import requests, zipfile
import gdown

TRAIN_ZIP_URL = 'https://drive.usercontent.google.com/download?id=1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16&export=download&authuser=0'
TEST_ZIP_URL  = 'https://drive.usercontent.google.com/download?id=12QrDrLT1F-X7UycvOoApXFqxTw3Zx93K&export=download&authuser=0'

train_zip = DATA_DIR / 'train.zip'
test_zip  = DATA_DIR / 'test.zip'


def _extract_id(url: str):
    try:
        return url.split('id=')[1].split('&')[0]
    except Exception:
        return None


def _ensure_zip(url: str, dest: Path):
    # Сначала обычная загрузка
    if not dest.exists():
        console.print(f'[cyan]Скачиваю {dest.name}...[/cyan]')
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(dest, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1<<20):
                    if chunk:
                        f.write(chunk)
        console.print(f'[green]Готово: {dest}[/green]')
    # Проверим, zip ли это
    if not zipfile.is_zipfile(dest):
        console.print(f'[yellow]Файл {dest.name} не похож на ZIP, пробую gdown...[/yellow]')
        file_id = _extract_id(url)
        if file_id:
            gdown.download(id=file_id, output=str(dest), quiet=False)
        else:
            gdown.download(url=url, output=str(dest), quiet=False, fuzzy=True)
    if not zipfile.is_zipfile(dest):
        raise RuntimeError(f'Не удалось получить валидный ZIP: {dest}')


def _safe_unzip(zip_path: Path, target_dir: Path, title: str):
    console.print(f'[bold cyan]Распаковываю {title}...[/bold cyan]')
    try:
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(target_dir)
    except zipfile.BadZipFile:
        console.print(f'[yellow]Поврежденный ZIP, повторная загрузка через gdown...[/yellow]')
        file_id = _extract_id(TRAIN_ZIP_URL if 'train' in zip_path.name else TEST_ZIP_URL)
        gdown.download(id=file_id, output=str(zip_path), quiet=False)
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(target_dir)


if not TRAIN_DIR.exists():
    _ensure_zip(TRAIN_ZIP_URL, train_zip)
    _safe_unzip(train_zip, DATA_DIR, 'train')

if not TEST_DIR.exists():
    _ensure_zip(TEST_ZIP_URL, test_zip)
    _safe_unzip(test_zip, DATA_DIR, 'test')

console.print('[bold green]Готово. Структура:[/bold green]')
for p in sorted(DATA_DIR.glob('*')):
    console.print(' -', p)


Файл train.zip не похож на ZIP, пробую gdown...

Downloading...
From (original): https://drive.google.com/uc?id=1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16
From (redirected): https://drive.google.com/uc?id=1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16&confirm=t&uuid=22126ad0-c5f6-4c84-844c-84c469ff458a
To: /content/data/train.zip
100%|██████████| 2.28G/2.28G [00:11<00:00, 207MB/s]


Распаковываю train...

Скачиваю test.zip...

Готово: /content/data/test.zip

...

Готово. Структура:

- /content/data/test.zip

- /content/data/test_kaggle

- /content/data/train

- /content/data/train.zip

In [8]:
# --- Детекция лица: MTCNN и OpenCV DNN ---

import requests

def get_mtcnn_detector():
    return MTCNN()

# Pretrained OpenCV DNN (res10_300x300_ssd)
PROTO_URL = 'https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt'
CAFFE_MODEL_URL = 'https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel'

DNN_DIR = DATA_DIR / 'cv_dnn'
DNN_DIR.mkdir(exist_ok=True)
PROTO_PATH = DNN_DIR / 'deploy.prototxt'
MODEL_PATH = DNN_DIR / 'res10_300x300_ssd_iter_140000.caffemodel'

def _download(url: str, dest: Path):
    if dest.exists():
        return
    console.print(f"[cyan]Скачиваю {dest.name}...[/cyan]")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(dest, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
    console.print(f"[green]Готово: {dest}[/green]")

_download(PROTO_URL, PROTO_PATH)
_download(CAFFE_MODEL_URL, MODEL_PATH)

def get_cv_dnn_detector():
    return cv2.dnn.readNetFromCaffe(str(PROTO_PATH), str(MODEL_PATH))


def detect_and_crop_face(image_bgr: np.ndarray, detector_type: str='mtcnn'):
    h, w = image_bgr.shape[:2]
    if detector_type == 'mtcnn':
        detector = get_mtcnn_detector()
        faces = detector.detect_faces(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
        if not faces:
            return None
        x, y, width, height = faces[0]['box']
        x, y = max(0, x), max(0, y)
        return image_bgr[y:y+height, x:x+width]
    else:
        net = get_cv_dnn_detector()
        blob = cv2.dnn.blobFromImage(image_bgr, 1.0, (300, 300), (104.0, 177.0, 123.0), swapRB=True)
        net.setInput(blob)
        detections = net.forward()
        conf_threshold = 0.5
        best = None
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > conf_threshold:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype('int')
                best = (max(0, x1), max(0, y1), min(w, x2)-x1, min(h, y2)-y1)
                break
        if best is None:
            return None
        x, y, width, height = best
        return image_bgr[y:y+height, x:x+width]


Скачиваю deploy.prototxt...

Готово: /content/data/cv_dnn/deploy.prototxt

Скачиваю res10_300x300_ssd_iter_140000.caffemodel...

Готово: /content/data/cv_dnn/res10_300x300_ssd_iter_140000.caffemodel

In [9]:
# --- Подготовка списков файлов и train/val split ---

def collect_dataset(train_dir: Path):
    image_paths, labels = [], []
    for class_idx, class_name in enumerate(CLASS_NAMES):
        class_dir = train_dir / class_name
        for p in class_dir.glob('*.jpg'):
            image_paths.append(str(p))
            labels.append(class_idx)
    return pd.DataFrame({'path': image_paths, 'label': labels})

train_df = collect_dataset(TRAIN_DIR)
train_df = train_df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)

tr_df, val_df = train_test_split(train_df, test_size=0.15, stratify=train_df['label'], random_state=SEED)
tr_df = tr_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

console.print('[bold]Train[/bold]:', tr_df.shape, '  [bold]Val[/bold]:', val_df.shape)
tr_df.head()


Train:
(42539, 2)
  Val:
(7508, 2)

,path,label
0,/content/data/train/fear/1485.jpg,3
1,/content/data/train/neutral/2735.jpg,5
2,/content/data/train/uncertain/178.jpg,8
3,/content/data/train/fear/3581.jpg,3
4,/content/data/train/sad/1422.jpg,6


In [ ]:
# --- Аугментации и препроцессинг ---

train_transform = A.Compose([
    A.LongestMaxSize(max_size=IMG_SIZE, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_REFLECT_101),
    A.OneOf([
        A.RandomBrightnessContrast(0.15, 0.15, p=1.0),
        A.CLAHE(clip_limit=2.0, p=1.0),
    ], p=0.5),
    A.HorizontalFlip(p=0.5),
    A.Affine(
        scale=(0.9, 1.1), translate_percent=(-0.02, 0.02), rotate=(-10, 10),
        shear=(-5, 5),
        mode=cv2.BORDER_REFLECT_101, cval=0, p=0.5
    ),
    A.CoarseDropout(
        min_holes=4, max_holes=4,
        min_height=IMG_SIZE // 32, min_width=IMG_SIZE // 32,
        max_height=IMG_SIZE // 16, max_width=IMG_SIZE // 16,
        fill_value=0, p=0.3
    ),
])

val_transform = A.Compose([
    A.LongestMaxSize(max_size=IMG_SIZE, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_REFLECT_101),
])


def read_image_and_face_crop(path: str):
    bgr = cv2.imread(path)
    if bgr is None:
        raise ValueError(f'Не удалось прочитать: {path}')
    face = detect_and_crop_face(bgr, FACE_DETECTOR)
    if face is None:
        face = bgr
    return face


def preprocess_image(path: str, is_train: bool):
    face = read_image_and_face_crop(path)
    if is_train:
        aug = train_transform(image=face)
    else:
        aug = val_transform(image=face)
    image = aug['image']
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0
    return image


def build_tf_dataset(df: pd.DataFrame, is_train: bool):
    paths = df['path'].values
    labels = df['label'].values

    def _py_map(path, label):
        path = path.decode()
        image = preprocess_image(path, is_train)
        return image, np.int32(label)

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if is_train:
        ds = ds.shuffle(4096, seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(lambda p, y: tf.numpy_function(_py_map, [p, y], [tf.float32, tf.int32]), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(lambda x, y: (tf.ensure_shape(x, [IMG_SIZE, IMG_SIZE, 3]), tf.ensure_shape(tf.cast(y, tf.int32), [])))
    ds = ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = build_tf_dataset(tr_df, True)
val_ds   = build_tf_dataset(val_df, False)


/tmp/ipython-input-3730348829.py:11: UserWarning: Argument(s) 'cval, mode' are not valid for transform Affine
  A.Affine(scale=(0.9, 1.1), translate_percent=(-0.02, 0.02), rotate=(-10, 10),


AttributeError: module 'albumentations' has no attribute 'Cutout'

In [ ]:
# --- Модель: EfficientNet-Lite0 с дообучением ---

# Используем легкую архитектуру для on-device
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Заморозим основу на раннем этапе, затем разморозим для fine-tune
base_model.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input')
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D(name='gap')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(N_CLASSES, activation='softmax', name='output')(x)
model = keras.Model(inputs, outputs)

# Под drop_remainder=True используем целочисленные шаги
steps_per_epoch = len(tr_df) // BATCH_SIZE
validation_steps = len(val_df) // BATCH_SIZE

optimizer = keras.optimizers.AdamW(learning_rate=BASE_LR, weight_decay=WEIGHT_DECAY)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

ckpt_dir = BASE_DIR / 'checkpoints'
ckpt_dir.mkdir(exist_ok=True)
ckpt_path = str(ckpt_dir / 'model.{epoch:02d}-{val_accuracy:.4f}.h5')

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath=ckpt_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
]

console.print('[bold cyan]Этап 1: дообучение головы...[/bold cyan]')
hist1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS//2,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
)

# Fine-tune: разморозим часть слоев
console.print('[bold cyan]Этап 2: fine-tune бэкбона...[/bold cyan]')
base_model.trainable = True
for layer in base_model.layers[:150]:
    layer.trainable = False
optimizer_finetune = keras.optimizers.AdamW(learning_rate=BASE_LR/10, weight_decay=WEIGHT_DECAY)
model.compile(optimizer=optimizer_finetune, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
hist2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS//2,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
)

history = {}
for k in hist1.history:
    history[k] = hist1.history[k] + hist2.history.get(k, [])

console.print('[bold green]Обучение завершено[/bold green]')


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Этап 1: дообучение головы...

Epoch 1/10


ValueError: Cannot take the length of shape with unknown rank.

In [ ]:
# --- Визуализация истории обучения ---

def plot_history(hist):
    fig, axs = plt.subplots(1, 2, figsize=(14,5))
    axs[0].plot(hist['loss'], label='train_loss')
    axs[0].plot(hist['val_loss'], label='val_loss')
    axs[0].legend(); axs[0].set_title('Loss'); axs[0].grid(True)
    axs[1].plot(hist['accuracy'], label='train_acc')
    axs[1].plot(hist['val_accuracy'], label='val_acc')
    axs[1].legend(); axs[1].set_title('Accuracy'); axs[1].grid(True)
    plt.show()

plot_history(history)


In [ ]:
# --- Оценка на валидации: отчеты и матрица ошибок ---

val_images, val_labels = [], []
for path, label in val_df[['path','label']].values:
    val_images.append(preprocess_image(path, is_train=False))
    val_labels.append(label)
val_images = np.stack(val_images)
val_labels = np.array(val_labels)

val_probs = model.predict(val_images, batch_size=BATCH_SIZE)
val_pred = val_probs.argmax(axis=1)

# Классификационный отчет
report = classification_report(val_labels, val_pred, target_names=CLASS_NAMES, digits=4)
print(report)

# Матрица ошибок
cm = confusion_matrix(val_labels, val_pred)
plt.figure(figsize=(9,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.title('Confusion Matrix')
plt.ylabel('True'); plt.xlabel('Predicted');
plt.show()

# ROC-AUC по классам (one-vs-rest)
y_true_bin = label_binarize(val_labels, classes=list(range(N_CLASSES)))
try:
    roc_auc = roc_auc_score(y_true_bin, val_probs, average=None)
    for i, c in enumerate(CLASS_NAMES):
        print(f'ROC-AUC[{c}] = {roc_auc[i]:.4f}')
    print(f'Macro ROC-AUC = {roc_auc.mean():.4f}')
except ValueError:
    print('ROC-AUC не посчитан (возможно один класс не предсказан)')


In [ ]:
# --- Grad-CAM визуализация важных областей ---
from tensorflow.keras.preprocessing import image as kimage

last_conv_layer_name = None
for layer in reversed(model.layers):
    if isinstance(layer, layers.Conv2D):
        last_conv_layer_name = layer.name
        break

if last_conv_layer_name is None:
    # Возьмем последнюю conv из base_model
    for layer in reversed(base_model.layers):
        if isinstance(layer, layers.Conv2D):
            last_conv_layer_name = layer.name
            break

import tensorflow.keras.backend as K

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Визуализируем несколько карт
num_show = min(6, len(val_images))
plt.figure(figsize=(12, 8))
for i in range(num_show):
    img = val_images[i]
    inp = np.expand_dims(img, 0)
    heatmap = make_gradcam_heatmap(inp, model, last_conv_layer_name)
    heatmap = cv2.resize(heatmap, (IMG_SIZE, IMG_SIZE))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted((img*255).astype(np.uint8), 0.6, heatmap, 0.4, 0)
    plt.subplot(2, (num_show+1)//2, i+1)
    plt.axis('off')
    plt.title(CLASS_NAMES[val_labels[i]])
    plt.imshow(overlay)
plt.suptitle('Grad-CAM важные области')
plt.tight_layout()
plt.show()


In [ ]:
# --- Экспорт в TFLite и ONNX ---
export_dir = BASE_DIR / 'export'
export_dir.mkdir(exist_ok=True)

# Сохранение Keras
keras_path = export_dir / 'fer_model.h5'
model.save(keras_path)
console.print('[green]Сохранено:[/green]', keras_path)

# TFLite float16
if DO_TFLITE_FP16:
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    tflite_model = converter.convert()
    tflite_fp16_path = export_dir / 'fer_model_fp16.tflite'
    with open(tflite_fp16_path, 'wb') as f:
        f.write(tflite_model)
    console.print('[green]TFLite FP16:[/green]', tflite_fp16_path)

# TFLite int8 (post-training quantization) с калибровкой на подвыборке
if DO_TFLITE_INT8:
    def representative_dataset():
        for i in range(100):
            idx = np.random.randint(0, len(tr_df))
            img = preprocess_image(tr_df.iloc[idx]['path'], is_train=False)
            yield [np.expand_dims(img.astype(np.float32), 0)]
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_int8 = converter.convert()
    tflite_int8_path = export_dir / 'fer_model_int8.tflite'
    with open(tflite_int8_path, 'wb') as f:
        f.write(tflite_int8)
    console.print('[green]TFLite INT8:[/green]', tflite_int8_path)

# ONNX
if DO_ONNX:
    import tf2onnx
    spec = (tf.TensorSpec((None, IMG_SIZE, IMG_SIZE, 3), tf.float32, name='input'),)
    onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
    onnx_path = export_dir / 'fer_model.onnx'
    with open(onnx_path, 'wb') as f:
        f.write(onnx_model.SerializeToString())
    console.print('[green]ONNX:[/green]', onnx_path)


In [ ]:
## Интеграция в Kotlin Multiplatform (Android/iOS)

- Экспортируйте TFLite-модель `fer_model_fp16.tflite` (или `fer_model_int8.tflite` для ускорения)
- На Android используйте `org.tensorflow:tensorflow-lite` и `tensorflow-lite-select-tf-ops` при необходимости
- На iOS можно использовать TensorFlowLiteSwift или переложить ONNX в CoreML через coremltools
- Предобработка на устройстве: MTCNN сравнительно тяжелый; на проде используйте MediaPipe Face Detection (BlazeFace) для реального времени, затем кроп лица → ресайз до 224x224 → нормализация 0..1 → TFLite-инференс

Пример псевдокода (Android, TFLite):


In [ ]:
// val tfliteModel = FileUtil.loadMappedFile(context, "fer_model_fp16.tflite")
// val interpreter = Interpreter(tfliteModel, Interpreter.Options().apply {
//     setNumThreads(4)
// })
// val input = ByteBuffer.allocateDirect(1 * 224 * 224 * 3 * 4).order(ByteOrder.nativeOrder())
// val output = ByteBuffer.allocateDirect(1 * 9 * 4).order(ByteOrder.nativeOrder())
// preprocess(faceBitmap, input) // resize 224x224, float32 0..1
// interpreter.run(input, output)
// val probs = FloatArray(9)
// output.asFloatBuffer().get(probs)
// val emotionIdx = probs.indices.maxBy { probs[it] }


In [ ]:
# --- Экспорт метрик и артефактов ---
metrics = {
    'classes': CLASS_NAMES,
    'val_accuracy': float(history['val_accuracy'][-1]) if len(history['val_accuracy'])>0 else None,
}
metrics_path = export_dir / 'metrics.json'
with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)
console.print('[green]Сохранены метрики:[/green]', metrics_path)

# t-SNE визуализация эмбеддингов
feat_model = keras.Model(model.input, model.get_layer('gap').output)
embeds = feat_model.predict(val_images, batch_size=BATCH_SIZE)
tsne = TSNE(n_components=2, random_state=SEED, init='pca', learning_rate='auto')
emb2d = tsne.fit_transform(embeds)

plt.figure(figsize=(8,6))
for i, c in enumerate(CLASS_NAMES):
    idx = np.where(val_labels==i)[0]
    plt.scatter(emb2d[idx,0], emb2d[idx,1], s=10, label=c)
plt.legend(ncol=3)
plt.title('t-SNE feature embeddings (val)')
plt.grid(True)
plt.show()
